**Connect google drive**

In [1]:
# Check for Nvidia GPU on colab
!nvidia-smi

Wed Jun 30 21:11:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#mount google drive to get the dataset and store weights (folder name - potato_yolov3)
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [3]:
#clone darknet
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15185, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 15185 (delta 57), reused 77 (delta 40), pack-reused 15073
Receiving objects: 100% (15185/15185), 13.55 MiB | 24.52 MiB/s, done.
Resolving deltas: 100% (10302/10302), done.


In [4]:
# compile darknet - change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

In [5]:
#configure darknet for yolov3
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [7]:
!echo "Potato" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/potato_yolov3' > data/obj.data
!mkdir data/obj

In [8]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-06-30 21:17:45--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  43.8MB/s    in 3.9s    

2021-06-30 21:17:50 (39.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [9]:
#extract images + annotations from training dataset
!unzip /mydrive/potato_yolov3/images.zip -d data/obj

Archive:  /mydrive/potato_yolov3/images.zip
  inflating: data/obj/train_dataset/Image1.jpg  
  inflating: data/obj/train_dataset/Image1.txt  
  inflating: data/obj/train_dataset/Image10.jpg  
  inflating: data/obj/train_dataset/Image10.txt  
  inflating: data/obj/train_dataset/Image100.jpg  
  inflating: data/obj/train_dataset/Image100.txt  
  inflating: data/obj/train_dataset/Image101.jpg  
  inflating: data/obj/train_dataset/Image101.txt  
  inflating: data/obj/train_dataset/Image102.jpg  
  inflating: data/obj/train_dataset/Image102.txt  
  inflating: data/obj/train_dataset/Image103.jpg  
  inflating: data/obj/train_dataset/Image103.txt  
  inflating: data/obj/train_dataset/Image104.jpg  
  inflating: data/obj/train_dataset/Image104.txt  
  inflating: data/obj/train_dataset/Image105.jpg  
  inflating: data/obj/train_dataset/Image105.txt  
  inflating: data/obj/train_dataset/Image106.jpg  
  inflating: data/obj/train_dataset/Image106.txt  
  inflating: data/obj/train_dataset/Image107

In [10]:
#get the list of images
import glob
images_list = glob.glob("data/obj/train_dataset/*.jpg")
print(images_list)

['data/obj/train_dataset/Image53.jpg', 'data/obj/train_dataset/Image99.jpg', 'data/obj/train_dataset/Image150.jpg', 'data/obj/train_dataset/Image115.jpg', 'data/obj/train_dataset/Image105.jpg', 'data/obj/train_dataset/Image28.jpg', 'data/obj/train_dataset/Image151.jpg', 'data/obj/train_dataset/Image98.jpg', 'data/obj/train_dataset/Image218.jpg', 'data/obj/train_dataset/Image61.jpg', 'data/obj/train_dataset/Image103.jpg', 'data/obj/train_dataset/Image208.jpg', 'data/obj/train_dataset/Image104.jpg', 'data/obj/train_dataset/Image154.jpg', 'data/obj/train_dataset/Image194.jpg', 'data/obj/train_dataset/Image106.jpg', 'data/obj/train_dataset/Image125.jpg', 'data/obj/train_dataset/Image71.jpg', 'data/obj/train_dataset/Image205.jpg', 'data/obj/train_dataset/Image36.jpg', 'data/obj/train_dataset/Image57.jpg', 'data/obj/train_dataset/Image217.jpg', 'data/obj/train_dataset/Image207.jpg', 'data/obj/train_dataset/Image128.jpg', 'data/obj/train_dataset/Image161.jpg', 'data/obj/train_dataset/Image21.

In [11]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Train YoloV3
#weights wil be stored in the google drive folder (/mydrive/potato_yolov3)
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Sho